In [1]:
import numpy as np
import pandas as pd
import os

In [ ]:
#Get all CSV files in data folder in list csvs & sort them
csvs = [f for f in os.listdir('data') if f.endswith('.csv')]
#Sort csvs
csvs.sort()
csvs

In [ ]:
#Read all csvs in one dataframe
df = pd.concat([pd.read_csv("data/"+csv, usecols=['ActionGeo_Lat', 'Actor1Code','Actor2Code','GoldsteinScale','SQLDATE','ActionGeo_Long']) for csv in csvs])
# df = pd.read_csv('goldstein_index_1990.csv')
#Convert SQLDATE to datetime to have year
df['SQLDATE'] = pd.to_datetime(df['SQLDATE'], format='%Y%m%d')
#Add year column
df['year'] = df['SQLDATE'].dt.year
#Remove rows where Actor1Code & Actor2Code is not COP, GOV, INS, JUD, MIL, OPP, REB, SEP, SPY, UAF, AGR, BUS, CRM, CVL, DEV, EDU, ELI, ENV, HLH, HRI, LAB, LEG, MED, REF, MAD, RAD, IGO, IMG, INT, MNC, NGM, NGO, UIS & reset index
df = df[df['Actor1Code'].isin(['COP', 'GOV', 'INS', 'JUD', 'MIL', 'OPP', 'REB', 'SEP', 'SPY', 'UAF', 'AGR', 'BUS', 'CRM', 'CVL', 'DEV', 'EDU', 'ELI', 'ENV', 'HLH', 'HRI', 'LAB', 'LEG', 'MED', 'REF', 'MAD', 'RAD', 'IGO', 'IMG', 'INT', 'MNC', 'NGM', 'NGO', 'UIS']) & df['Actor2Code'].isin(['COP', 'GOV', 'INS', 'JUD', 'MIL', 'OPP', 'REB', 'SEP', 'SPY', 'UAF', 'AGR', 'BUS', 'CRM', 'CVL', 'DEV', 'EDU', 'ELI', 'ENV', 'HLH', 'HRI', 'LAB', 'LEG', 'MED', 'REF', 'MAD', 'RAD', 'IGO', 'IMG', 'INT', 'MNC', 'NGM', 'NGO', 'UIS'])].reset_index(drop=True)

In [ ]:
def lat_long_to_PG_lat_long(x, list_lat_long):
    for i in range(len(x)):
        if x[i] >= 0:
            if x[i] % 1 >= 0 and x[i] % 1 <= 0.5:
                list_lat_long.append(int(x[i]) + 0.25)
            else:
                list_lat_long.append(int(x[i]) + 0.75)

        if x[i] < 0:
            if x[i] % 1 >= 0 and x[i] % 1 <= 0.5:
                list_lat_long.append(int(x[i]) - 0.75)
            else:
                list_lat_long.append(int(x[i]) - 0.25)


pg_lat = []
pg_long = []
lat_long_to_PG_lat_long(df["ActionGeo_Lat"], pg_lat)
lat_long_to_PG_lat_long(df["ActionGeo_Long"], pg_long)
df['pg_lat'] = pg_lat
df['pg_long'] = pg_long
df

In [ ]:
df.to_csv('goldstein_index_1990_to_2022_raw.csv', index=False)

In [2]:
df = pd.read_csv('goldstein_index_1990_to_2022_raw.csv')

In [3]:
df['SQLDATE']

0           1990-01-01
1           1990-01-01
2           1990-01-01
3           1990-01-01
4           1990-01-01
               ...    
91973757    2022-09-30
91973758    2022-09-30
91973759    2022-09-30
91973760    2022-09-30
91973761    2022-09-30
Name: SQLDATE, Length: 91973762, dtype: object

In [4]:
#Assign SQLDATE as date
df['SQLDATE'] = pd.to_datetime(df['SQLDATE'], infer_datetime_format=True)
#Create new column month_year from SQLDATE & drop year column
df['month_year'] = df['SQLDATE'].dt.to_period('M')
df = df.drop(['year'], axis=1)
df

,SQLDATE,Actor1Code,Actor2Code,GoldsteinScale,ActionGeo_Lat,ActionGeo_Long,pg_lat,pg_long,month_year
0,1990-01-01,BUS,COP,-4.4,51.4500,5.46667,51.25,5.25,1990-01
1,1990-01-01,BUS,JUD,3.0,42.2373,-71.53140,42.25,-71.75,1990-01
2,1990-01-01,BUS,MIL,-9.0,0.0000,0.00000,0.25,0.25,1990-01
3,1990-01-01,BUS,REB,6.0,0.0000,0.00000,0.25,0.25,1990-01
4,1990-01-01,BUS,REB,7.4,0.0000,0.00000,0.25,0.25,1990-01
...,...,...,...,...,...,...,...,...,...
91973757,2022-09-30,MIL,CVL,1.0,38.8990,-77.04220,38.75,-77.25,2022-09
91973758,2022-09-30,MIL,CVL,1.0,56.0000,10.00000,56.25,10.25,2022-09
91973759,2022-09-30,OPP,EDU,7.0,32.0000,53.00000,32.25,53.25,2022-09
91973760,2022-09-30,OPP,GOV,-2.0,25.3833,83.01670,25.25,83.25,2022-09


In [5]:
#Pivot df to place Actor1Code as columns. Get mean and count of GoldsteinScale 
df_pivot_1 = df.pivot_table(index=['month_year', 'pg_lat', 'pg_long'], columns='Actor1Code', values=['GoldsteinScale'], aggfunc=['mean', 'count'])
df_pivot_1 = df_pivot_1.reset_index().reset_index(drop=True)
df_pivot_1

month_year pg_lat pg_long           mean                          \
                                     GoldsteinScale                           
Actor1Code                                      AGR BUS  COP CRM   CVL  EDU   
0             1990-01 -41.75  174.25            NaN NaN  NaN NaN   NaN  NaN   
1             1990-01 -35.25  174.25            NaN NaN  0.0 NaN   NaN  NaN   
2             1990-01 -34.75  -66.25            NaN NaN  7.0 NaN   NaN  NaN   
3             1990-01 -34.75  -58.75            NaN NaN  NaN NaN  -5.5  NaN   
4             1990-01 -34.75  138.75            NaN NaN  NaN NaN   NaN  NaN   
...               ...    ...     ...            ...  ..  ...  ..   ...  ...   
1439935       2022-09  72.25  -40.75           0.95 NaN  NaN NaN   1.9 -1.3   
1439936       2022-09  73.75   80.75            NaN NaN  NaN NaN   NaN  NaN   
1439937       2022-09  74.25   86.75            NaN NaN  NaN NaN -10.0  NaN   
1439938       2022-09  79.25   98.25            NaN NaN  NaN NaN   NaN  NaN   
1439939       2022-09  82.75  -81.25            NaN NaN  NaN NaN   0.0  NaN   

                ...          count                                       
                ... GoldsteinScale                                       
Actor1Code ELI  ...            MNC NGO OPP RAD  REB REF SEP SPY UAF UIS  
0          NaN  ...            NaN NaN NaN NaN  1.0 NaN NaN NaN NaN NaN  
1          NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  
2          NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  
3          NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  
4          NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  
...         ..  ...            ...  ..  ..  ..  ...  ..  ..  ..  ..  ..  
1439935    NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  
1439936    NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  
1439937    NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  
1439938    NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  
1439939    NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  

[1439940 rows x 63 columns]

In [6]:
# Make only one header row for df_pivot_1 and remove the multi-index
df_pivot_1.columns = ['_'.join(col).strip() for col in df_pivot_1.columns.values]

#Remove all underscores if they are at the end of the column name
df_pivot_1.columns = df_pivot_1.columns.str.rstrip('_')
df_pivot_1

,month_year,pg_lat,pg_long,mean_GoldsteinScale_AGR,mean_GoldsteinScale_BUS,mean_GoldsteinScale_COP,mean_GoldsteinScale_CRM,mean_GoldsteinScale_CVL,mean_GoldsteinScale_EDU,mean_GoldsteinScale_ELI,...,count_GoldsteinScale_MNC,count_GoldsteinScale_NGO,count_GoldsteinScale_OPP,count_GoldsteinScale_RAD,count_GoldsteinScale_REB,count_GoldsteinScale_REF,count_GoldsteinScale_SEP,count_GoldsteinScale_SPY,count_GoldsteinScale_UAF,count_GoldsteinScale_UIS
0,1990-01,-41.75,174.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,1990-01,-35.25,174.25,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-01,-34.75,-66.25,NaN,NaN,7.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-01,-34.75,-58.75,NaN,NaN,NaN,NaN,-5.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-01,-34.75,138.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439935,2022-09,72.25,-40.75,0.95,NaN,NaN,NaN,1.9,-1.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439936,2022-09,73.75,80.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439937,2022-09,74.25,86.75,NaN,NaN,NaN,NaN,-10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439938,2022-09,79.25,98.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
prio_grid = pd.read_csv('../PRIO GRID spine.csv') #read in PRIO GRID spine file

#Create date_df with month-year column containing 1990 to 2022
date_df = pd.DataFrame()
date_df['month_year'] = pd.date_range(start='1990-01-01', end='2022-12-01', freq='MS').to_period('M')
date_df['key'] = 1
prio_grid['key'] = 1
base_df = pd.merge(date_df, prio_grid, on ='key').drop("key", 1)
base_df

,month_year,gid,lon,lat
0,1990-01,49182,-69.25,-55.75
1,1990-01,49183,-68.75,-55.75
2,1990-01,49184,-68.25,-55.75
3,1990-01,49185,-67.75,-55.75
4,1990-01,49186,-67.25,-55.75
...,...,...,...,...
25667923,2022-12,249340,-70.25,83.25
25667924,2022-12,249341,-69.75,83.25
25667925,2022-12,249342,-69.25,83.25
25667926,2022-12,249343,-68.75,83.25


In [8]:
#Merge GoldsteinScale column from lab_bus with base_df on pg_lat and pg_long into one new dataframe df
df_pivot_1 = pd.merge(base_df, df_pivot_1, how = 'left', left_on = ['lat','lon','month_year'], right_on = ['pg_lat','pg_long','month_year']).drop(['pg_lat','pg_long'], 1) #merge lab_bus with base_df
# df_final = pd.merge(df_temp[['GLOBALEVENTID','pg_lat','pg_long','Actor1Code','Actor2Code','GoldsteinScale','SQLDATE']],base_df, how = 'left', left_on = ['pg_lat','pg_long','SQLDATE'], right_on = ['lat','lon','date']).drop(['pg_lat','pg_long','SQLDATE'], 1) #merge lab_bus with base_df

In [9]:
df_pivot_1

,month_year,gid,lon,lat,mean_GoldsteinScale_AGR,mean_GoldsteinScale_BUS,mean_GoldsteinScale_COP,mean_GoldsteinScale_CRM,mean_GoldsteinScale_CVL,mean_GoldsteinScale_EDU,...,count_GoldsteinScale_MNC,count_GoldsteinScale_NGO,count_GoldsteinScale_OPP,count_GoldsteinScale_RAD,count_GoldsteinScale_REB,count_GoldsteinScale_REF,count_GoldsteinScale_SEP,count_GoldsteinScale_SPY,count_GoldsteinScale_UAF,count_GoldsteinScale_UIS
0,1990-01,49182,-69.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-01,49183,-68.75,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-01,49184,-68.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-01,49185,-67.75,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-01,49186,-67.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25667923,2022-12,249340,-70.25,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667924,2022-12,249341,-69.75,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667925,2022-12,249342,-69.25,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667926,2022-12,249343,-68.75,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#Create new column mean_Goldstein_BUS_MNC as mean of GoldsteinScale_BUS and GoldsteinScale_MNC
try:
    df_pivot_1['mean_GoldsteinScale_BUS_MNC'] = df_pivot_1[['mean_GoldsteinScale_BUS','mean_GoldsteinScale_MNC']].mean(axis=1)
    #Create new column count_Goldstein_BUS_MNC as sum of count_GoldsteinScale_BUS and count_GoldsteinScale_MNC
    df_pivot_1['count_GoldsteinScale_BUS_MNC'] = df_pivot_1[['count_GoldsteinScale_BUS','count_GoldsteinScale_MNC']].sum(axis=1)
except:
    print('BUS or MNC not in Actor1Code')

try:
    #Ngm_ngo
    df_pivot_1['mean_GoldsteinScale_NGM_NGO'] = df_pivot_1[['mean_GoldsteinScale_NGM','mean_GoldsteinScale_NGO']].mean(axis=1)
    df_pivot_1['count_GoldsteinScale_NGM_NGO'] = df_pivot_1[['count_GoldsteinScale_NGM','count_GoldsteinScale_NGO']].sum(axis=1)
except:
    print('NGM or NGO not in Actor1Code')

try:
    #Cop_jud_mil_spy
    df_pivot_1['mean_GoldsteinScale_COP_JUD_MIL_SPY'] = df_pivot_1[['mean_GoldsteinScale_COP','mean_GoldsteinScale_JUD','mean_GoldsteinScale_MIL','mean_GoldsteinScale_SPY']].mean(axis=1)
    df_pivot_1['count_GoldsteinScale_COP_JUD_MIL_SPY'] = df_pivot_1[['count_GoldsteinScale_COP','count_GoldsteinScale_JUD','count_GoldsteinScale_MIL','count_GoldsteinScale_SPY']].sum(axis=1)
except:
    print('COP, JUD, MIL or SPY not in Actor1Code')

try:
    #Ins_reb_img
    df_pivot_1['mean_GoldsteinScale_INS_REB_IMG'] = df_pivot_1[['mean_GoldsteinScale_INS','mean_GoldsteinScale_REB','mean_GoldsteinScale_IMG']].mean(axis=1)
    df_pivot_1['count_GoldsteinScale_INS_REB_IMG'] = df_pivot_1[['count_GoldsteinScale_INS','count_GoldsteinScale_REB','count_GoldsteinScale_IMG']].sum(axis=1)
except:
    print('INS, REB or IMG not in Actor1Code')

try:
    # Int_uis
    df_pivot_1['mean_GoldsteinScale_INT_UIS'] = df_pivot_1[['mean_GoldsteinScale_INT','mean_GoldsteinScale_UIS']].mean(axis=1)
    df_pivot_1['count_GoldsteinScale_INT_UIS'] = df_pivot_1[['count_GoldsteinScale_INT','count_GoldsteinScale_UIS']].sum(axis=1)
except:
    print('INT or UIS not in Actor1Code')

NGM or NGO not in Actor1Code


In [11]:
df_pivot_1

,month_year,gid,lon,lat,mean_GoldsteinScale_AGR,mean_GoldsteinScale_BUS,mean_GoldsteinScale_COP,mean_GoldsteinScale_CRM,mean_GoldsteinScale_CVL,mean_GoldsteinScale_EDU,...,count_GoldsteinScale_UAF,count_GoldsteinScale_UIS,mean_GoldsteinScale_BUS_MNC,count_GoldsteinScale_BUS_MNC,mean_GoldsteinScale_COP_JUD_MIL_SPY,count_GoldsteinScale_COP_JUD_MIL_SPY,mean_GoldsteinScale_INS_REB_IMG,count_GoldsteinScale_INS_REB_IMG,mean_GoldsteinScale_INT_UIS,count_GoldsteinScale_INT_UIS
0,1990-01,49182,-69.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
1,1990-01,49183,-68.75,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,1990-01,49184,-68.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,1990-01,49185,-67.75,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,1990-01,49186,-67.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25667923,2022-12,249340,-70.25,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
25667924,2022-12,249341,-69.75,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
25667925,2022-12,249342,-69.25,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
25667926,2022-12,249343,-68.75,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [12]:
#Rename columns beginning with mean_GoldsteinScale_ + {Actor1Code} to {Actor1Code}_All_Actors_GoldsteinScale
df_pivot_1 = df_pivot_1.rename(columns=lambda x: x.replace('mean_GoldsteinScale_', '') + '_All_Actors_GoldsteinScale' if x.startswith('mean_GoldsteinScale_') else x)

#Rename columns beginning with count_GoldsteinScale_ + {Actor1Code} to {Actor1Code}_All_Actors_Count
df_pivot_1 = df_pivot_1.rename(columns=lambda x: x.replace('count_GoldsteinScale_', '') + '_All_Actors_Count' if x.startswith('count_GoldsteinScale_') else x)

#Sort columns alphabetically
df_pivot_1 = df_pivot_1.reindex(sorted(df_pivot_1.columns), axis=1)
#Keep 'gid', 'lat', 'lon', 'year' at the beginning of the dataframe
cols = df_pivot_1.columns.tolist()
cols = cols[-4:] + cols[:-4]
df_pivot_1 = df_pivot_1[cols]

#Drop lat, lon
df_pivot_1 = df_pivot_1.drop(['lat','lon'], 1)

#Replace 0 with NaN in columns ending with _All_Actors_Count
df_pivot_1[df_pivot_1.columns[df_pivot_1.columns.str.endswith('_All_Actors_Count')]] = df_pivot_1[df_pivot_1.columns[df_pivot_1.columns.str.endswith('_All_Actors_Count')]].replace(0, np.nan)

df_pivot_1

,gid,month_year,AGR_All_Actors_Count,AGR_All_Actors_GoldsteinScale,BUS_All_Actors_Count,BUS_All_Actors_GoldsteinScale,BUS_MNC_All_Actors_Count,BUS_MNC_All_Actors_GoldsteinScale,COP_All_Actors_Count,COP_All_Actors_GoldsteinScale,...,REF_All_Actors_Count,REF_All_Actors_GoldsteinScale,SEP_All_Actors_Count,SEP_All_Actors_GoldsteinScale,SPY_All_Actors_Count,SPY_All_Actors_GoldsteinScale,UAF_All_Actors_Count,UAF_All_Actors_GoldsteinScale,UIS_All_Actors_Count,UIS_All_Actors_GoldsteinScale
0,49182,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49183,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49184,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49185,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49186,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25667923,249340,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667924,249341,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667925,249342,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667926,249343,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#Pivot df to place Actor2Code as columns. Get mean and count of GoldsteinScale 
df_pivot_2 = df.pivot_table(index=['month_year', 'pg_lat', 'pg_long'], columns='Actor2Code', values=['GoldsteinScale'], aggfunc=['mean', 'count'])
df_pivot_2 = df_pivot_2.reset_index().reset_index(drop=True)
df_pivot_2

month_year pg_lat pg_long           mean                            \
                                     GoldsteinScale                             
Actor2Code                                      AGR       BUS  COP  CRM   CVL   
0             1990-01 -41.75  174.25            NaN       NaN  NaN  NaN   NaN   
1             1990-01 -35.25  174.25            NaN       NaN  NaN  NaN   NaN   
2             1990-01 -34.75  -66.25            NaN       NaN  NaN  NaN   NaN   
3             1990-01 -34.75  -58.75            NaN       NaN -5.5  NaN   NaN   
4             1990-01 -34.75  138.75            NaN       NaN  NaN  NaN  -4.4   
...               ...    ...     ...            ...       ...  ...  ...   ...   
1439935       2022-09  72.25  -40.75            2.8  1.866667  NaN  1.3   2.8   
1439936       2022-09  73.75   80.75            NaN       NaN  NaN -2.0   NaN   
1439937       2022-09  74.25   86.75            NaN       NaN  NaN  NaN -10.0   
1439938       2022-09  79.25   98.25            NaN       NaN  NaN  NaN   NaN   
1439939       2022-09  82.75  -81.25            NaN       NaN  NaN  NaN   NaN   

                     ...          count                                        
                     ... GoldsteinScale                                        
Actor2Code  EDU ELI  ...            MNC NGO OPP RAD  REB REF SEP SPY  UAF UIS  
0           NaN NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN  NaN NaN  
1           NaN NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN  1.0 NaN  
2           NaN NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN  NaN NaN  
3           NaN NaN  ...            NaN NaN NaN NaN  2.0 NaN NaN NaN  NaN NaN  
4           NaN NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN  NaN NaN  
...         ...  ..  ...            ...  ..  ..  ..  ...  ..  ..  ..  ...  ..  
1439935     1.9 NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN  NaN NaN  
1439936     NaN NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN  NaN NaN  
1439937     NaN NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN  NaN NaN  
1439938    -2.0 NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN  NaN NaN  
1439939     0.0 NaN  ...            NaN NaN NaN NaN  NaN NaN NaN NaN  NaN NaN  

[1439940 rows x 63 columns]

In [14]:
# Make only one header row for df_pivot_2 and remove the multi-index
df_pivot_2.columns = ['_'.join(col).strip() for col in df_pivot_2.columns.values]

#Remove all underscores if they are at the end of the column name
df_pivot_2.columns = df_pivot_2.columns.str.rstrip('_')
df_pivot_2

,month_year,pg_lat,pg_long,mean_GoldsteinScale_AGR,mean_GoldsteinScale_BUS,mean_GoldsteinScale_COP,mean_GoldsteinScale_CRM,mean_GoldsteinScale_CVL,mean_GoldsteinScale_EDU,mean_GoldsteinScale_ELI,...,count_GoldsteinScale_MNC,count_GoldsteinScale_NGO,count_GoldsteinScale_OPP,count_GoldsteinScale_RAD,count_GoldsteinScale_REB,count_GoldsteinScale_REF,count_GoldsteinScale_SEP,count_GoldsteinScale_SPY,count_GoldsteinScale_UAF,count_GoldsteinScale_UIS
0,1990-01,-41.75,174.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-01,-35.25,174.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,1990-01,-34.75,-66.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-01,-34.75,-58.75,NaN,NaN,-5.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
4,1990-01,-34.75,138.75,NaN,NaN,NaN,NaN,-4.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439935,2022-09,72.25,-40.75,2.8,1.866667,NaN,1.3,2.8,1.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439936,2022-09,73.75,80.75,NaN,NaN,NaN,-2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439937,2022-09,74.25,86.75,NaN,NaN,NaN,NaN,-10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439938,2022-09,79.25,98.25,NaN,NaN,NaN,NaN,NaN,-2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#Merge GoldsteinScale column from lab_bus with base_df on pg_lat and pg_long into one new dataframe df
df_pivot_2 = pd.merge(base_df, df_pivot_2, how = 'left', left_on = ['lat','lon','month_year'], right_on = ['pg_lat','pg_long','month_year']).drop(['pg_lat','pg_long'], 1) #merge lab_bus with base_df
# df_final = pd.merge(df_temp[['GLOBALEVENTID','pg_lat','pg_long','Actor1Code','Actor2Code','GoldsteinScale','SQLDATE']],base_df, how = 'left', left_on = ['pg_lat','pg_long','SQLDATE'], right_on = ['lat','lon','date']).drop(['pg_lat','pg_long','SQLDATE'], 1) #merge lab_bus with base_df

In [16]:
df_pivot_2

,month_year,gid,lon,lat,mean_GoldsteinScale_AGR,mean_GoldsteinScale_BUS,mean_GoldsteinScale_COP,mean_GoldsteinScale_CRM,mean_GoldsteinScale_CVL,mean_GoldsteinScale_EDU,...,count_GoldsteinScale_MNC,count_GoldsteinScale_NGO,count_GoldsteinScale_OPP,count_GoldsteinScale_RAD,count_GoldsteinScale_REB,count_GoldsteinScale_REF,count_GoldsteinScale_SEP,count_GoldsteinScale_SPY,count_GoldsteinScale_UAF,count_GoldsteinScale_UIS
0,1990-01,49182,-69.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-01,49183,-68.75,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-01,49184,-68.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-01,49185,-67.75,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-01,49186,-67.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25667923,2022-12,249340,-70.25,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667924,2022-12,249341,-69.75,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667925,2022-12,249342,-69.25,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667926,2022-12,249343,-68.75,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#Create new column mean_Goldstein_BUS_MNC as mean of GoldsteinScale_BUS and GoldsteinScale_MNC
try:
    df_pivot_2['mean_GoldsteinScale_BUS_MNC'] = df_pivot_2[['mean_GoldsteinScale_BUS','mean_GoldsteinScale_MNC']].mean(axis=1)
    #Create new column count_Goldstein_BUS_MNC as sum of count_GoldsteinScale_BUS and count_GoldsteinScale_MNC
    df_pivot_2['count_GoldsteinScale_BUS_MNC'] = df_pivot_2[['count_GoldsteinScale_BUS','count_GoldsteinScale_MNC']].sum(axis=1)
except:
    print('BUS or MNC not in Actor1Code')

try:
    #Ngm_ngo
    df_pivot_2['mean_GoldsteinScale_NGM_NGO'] = df_pivot_2[['mean_GoldsteinScale_NGM','mean_GoldsteinScale_NGO']].mean(axis=1)
    df_pivot_2['count_GoldsteinScale_NGM_NGO'] = df_pivot_2[['count_GoldsteinScale_NGM','count_GoldsteinScale_NGO']].sum(axis=1)
except:
    print('NGM or NGO not in Actor1Code')

try:
    #Cop_jud_mil_spy
    df_pivot_2['mean_GoldsteinScale_COP_JUD_MIL_SPY'] = df_pivot_2[['mean_GoldsteinScale_COP','mean_GoldsteinScale_JUD','mean_GoldsteinScale_MIL','mean_GoldsteinScale_SPY']].mean(axis=1)
    df_pivot_2['count_GoldsteinScale_COP_JUD_MIL_SPY'] = df_pivot_2[['count_GoldsteinScale_COP','count_GoldsteinScale_JUD','count_GoldsteinScale_MIL','count_GoldsteinScale_SPY']].sum(axis=1)
except:
    print('COP, JUD, MIL or SPY not in Actor1Code')

try:
    #Ins_reb_img
    df_pivot_2['mean_GoldsteinScale_INS_REB_IMG'] = df_pivot_2[['mean_GoldsteinScale_INS','mean_GoldsteinScale_REB','mean_GoldsteinScale_IMG']].mean(axis=1)
    df_pivot_2['count_GoldsteinScale_INS_REB_IMG'] = df_pivot_2[['count_GoldsteinScale_INS','count_GoldsteinScale_REB','count_GoldsteinScale_IMG']].sum(axis=1)
except:
    print('INS, REB or IMG not in Actor1Code')

try:
    # Int_uis
    df_pivot_2['mean_GoldsteinScale_INT_UIS'] = df_pivot_2[['mean_GoldsteinScale_INT','mean_GoldsteinScale_UIS']].mean(axis=1)
    df_pivot_2['count_GoldsteinScale_INT_UIS'] = df_pivot_2[['count_GoldsteinScale_INT','count_GoldsteinScale_UIS']].sum(axis=1)
except:
    print('INT or UIS not in Actor1Code')

NGM or NGO not in Actor1Code


In [18]:
df_pivot_1

,gid,month_year,AGR_All_Actors_Count,AGR_All_Actors_GoldsteinScale,BUS_All_Actors_Count,BUS_All_Actors_GoldsteinScale,BUS_MNC_All_Actors_Count,BUS_MNC_All_Actors_GoldsteinScale,COP_All_Actors_Count,COP_All_Actors_GoldsteinScale,...,REF_All_Actors_Count,REF_All_Actors_GoldsteinScale,SEP_All_Actors_Count,SEP_All_Actors_GoldsteinScale,SPY_All_Actors_Count,SPY_All_Actors_GoldsteinScale,UAF_All_Actors_Count,UAF_All_Actors_GoldsteinScale,UIS_All_Actors_Count,UIS_All_Actors_GoldsteinScale
0,49182,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49183,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49184,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49185,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49186,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25667923,249340,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667924,249341,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667925,249342,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667926,249343,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_pivot_2

,month_year,gid,lon,lat,mean_GoldsteinScale_AGR,mean_GoldsteinScale_BUS,mean_GoldsteinScale_COP,mean_GoldsteinScale_CRM,mean_GoldsteinScale_CVL,mean_GoldsteinScale_EDU,...,count_GoldsteinScale_UAF,count_GoldsteinScale_UIS,mean_GoldsteinScale_BUS_MNC,count_GoldsteinScale_BUS_MNC,mean_GoldsteinScale_COP_JUD_MIL_SPY,count_GoldsteinScale_COP_JUD_MIL_SPY,mean_GoldsteinScale_INS_REB_IMG,count_GoldsteinScale_INS_REB_IMG,mean_GoldsteinScale_INT_UIS,count_GoldsteinScale_INT_UIS
0,1990-01,49182,-69.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
1,1990-01,49183,-68.75,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,1990-01,49184,-68.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,1990-01,49185,-67.75,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,1990-01,49186,-67.25,-55.75,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25667923,2022-12,249340,-70.25,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
25667924,2022-12,249341,-69.75,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
25667925,2022-12,249342,-69.25,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
25667926,2022-12,249343,-68.75,83.25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [20]:
#Rename columns beginning with mean_GoldsteinScale_ + {Actor1Code} to All_Actors_{Actor1Code}_+GoldsteinScale
df_pivot_2 = df_pivot_2.rename(columns=lambda x: x.replace('mean_GoldsteinScale_', 'All_Actors_') if x.startswith('mean_GoldsteinScale_') else x)
#Add _GoldsteinScale to the end of the column names beginning with All_Actors_
df_pivot_2 = df_pivot_2.rename(columns=lambda x: x + '_GoldsteinScale' if x.startswith('All_Actors_') else x)

#Rename columns beginning with count_GoldsteinScale_ + {Actor1Code} to All_Actors_{Actor1Code}_+Count
df_pivot_2 = df_pivot_2.rename(columns=lambda x: x.replace('count_GoldsteinScale_', 'All_Actors_') if x.startswith('count_GoldsteinScale_') else x)
#Add _Count to the end of the column names beginning with All_Actors_
df_pivot_2 = df_pivot_2.rename(columns=lambda x: x + '_Count' if x.startswith('All_Actors_') else x)

#Sort columns alphabetically
df_pivot_2 = df_pivot_2.reindex(sorted(df_pivot_2.columns), axis=1)
#Keep 'gid', 'lat', 'lon', 'year' at the beginning of the dataframe
cols = df_pivot_2.columns.tolist()
cols = cols[-4:] + cols[:-4]
df_pivot_2 = df_pivot_2[cols]

#Drop lat, lon
df_pivot_2 = df_pivot_2.drop(['lat','lon'], 1)

#Replace 0 with NaN in columns ending with _All_Actors_Count
df_pivot_2[df_pivot_2.columns[df_pivot_2.columns.str.endswith('_Count')]] = df_pivot_2[df_pivot_2.columns[df_pivot_2.columns.str.endswith('_Count')]].replace(0, np.nan)

df_pivot_2

,gid,month_year,All_Actors_AGR_Count,All_Actors_AGR_GoldsteinScale_Count,All_Actors_BUS_Count,All_Actors_BUS_GoldsteinScale_Count,All_Actors_BUS_MNC_Count,All_Actors_BUS_MNC_GoldsteinScale_Count,All_Actors_COP_Count,All_Actors_COP_GoldsteinScale_Count,...,All_Actors_REF_Count,All_Actors_REF_GoldsteinScale_Count,All_Actors_SEP_Count,All_Actors_SEP_GoldsteinScale_Count,All_Actors_SPY_Count,All_Actors_SPY_GoldsteinScale_Count,All_Actors_UAF_Count,All_Actors_UAF_GoldsteinScale_Count,All_Actors_UIS_Count,All_Actors_UIS_GoldsteinScale_Count
0,49182,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49183,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49184,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49185,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49186,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25667923,249340,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667924,249341,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667925,249342,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667926,249343,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_pivot_1.columns

Index(['gid', 'month_year', 'AGR_All_Actors_Count',
       'AGR_All_Actors_GoldsteinScale', 'BUS_All_Actors_Count',
       'BUS_All_Actors_GoldsteinScale', 'BUS_MNC_All_Actors_Count',
       'BUS_MNC_All_Actors_GoldsteinScale', 'COP_All_Actors_Count',
       'COP_All_Actors_GoldsteinScale', 'COP_JUD_MIL_SPY_All_Actors_Count',
       'COP_JUD_MIL_SPY_All_Actors_GoldsteinScale', 'CRM_All_Actors_Count',
       'CRM_All_Actors_GoldsteinScale', 'CVL_All_Actors_Count',
       'CVL_All_Actors_GoldsteinScale', 'EDU_All_Actors_Count',
       'EDU_All_Actors_GoldsteinScale', 'ELI_All_Actors_Count',
       'ELI_All_Actors_GoldsteinScale', 'ENV_All_Actors_Count',
       'ENV_All_Actors_GoldsteinScale', 'GOV_All_Actors_Count',
       'GOV_All_Actors_GoldsteinScale', 'HLH_All_Actors_Count',
       'HLH_All_Actors_GoldsteinScale', 'HRI_All_Actors_Count',
       'HRI_All_Actors_GoldsteinScale', 'IGO_All_Actors_Count',
       'IGO_All_Actors_GoldsteinScale', 'IMG_All_Actors_Count',
       'IMG_All_Acto

In [22]:
df_pivot_2.columns

Index(['gid', 'month_year', 'All_Actors_AGR_Count',
       'All_Actors_AGR_GoldsteinScale_Count', 'All_Actors_BUS_Count',
       'All_Actors_BUS_GoldsteinScale_Count', 'All_Actors_BUS_MNC_Count',
       'All_Actors_BUS_MNC_GoldsteinScale_Count', 'All_Actors_COP_Count',
       'All_Actors_COP_GoldsteinScale_Count',
       'All_Actors_COP_JUD_MIL_SPY_Count',
       'All_Actors_COP_JUD_MIL_SPY_GoldsteinScale_Count',
       'All_Actors_CRM_Count', 'All_Actors_CRM_GoldsteinScale_Count',
       'All_Actors_CVL_Count', 'All_Actors_CVL_GoldsteinScale_Count',
       'All_Actors_EDU_Count', 'All_Actors_EDU_GoldsteinScale_Count',
       'All_Actors_ELI_Count', 'All_Actors_ELI_GoldsteinScale_Count',
       'All_Actors_ENV_Count', 'All_Actors_ENV_GoldsteinScale_Count',
       'All_Actors_GOV_Count', 'All_Actors_GOV_GoldsteinScale_Count',
       'All_Actors_HLH_Count', 'All_Actors_HLH_GoldsteinScale_Count',
       'All_Actors_HRI_Count', 'All_Actors_HRI_GoldsteinScale_Count',
       'All_Actors_IGO_

In [24]:
del df

In [25]:
#Merge df_pivot_1 and df_pivot_2 on 'gid' and 'year' inner
df_final = pd.merge(df_pivot_1, df_pivot_2, on=['gid','month_year'], how='inner')

In [26]:
df_final

,gid,month_year,AGR_All_Actors_Count,AGR_All_Actors_GoldsteinScale,BUS_All_Actors_Count,BUS_All_Actors_GoldsteinScale,BUS_MNC_All_Actors_Count,BUS_MNC_All_Actors_GoldsteinScale,COP_All_Actors_Count,COP_All_Actors_GoldsteinScale,...,All_Actors_REF_Count,All_Actors_REF_GoldsteinScale_Count,All_Actors_SEP_Count,All_Actors_SEP_GoldsteinScale_Count,All_Actors_SPY_Count,All_Actors_SPY_GoldsteinScale_Count,All_Actors_UAF_Count,All_Actors_UAF_GoldsteinScale_Count,All_Actors_UIS_Count,All_Actors_UIS_GoldsteinScale_Count
0,49182,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49183,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49184,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49185,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49186,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25667923,249340,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667924,249341,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667925,249342,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25667926,249343,2022-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_final.to_csv('goldstein_index_1990_to_2022_monthly.csv', index=False)

: 

In [ ]:
print(df_final.columns)

In [ ]:
# df_final.to_hdf(r'goldstein_index_1990_final.h5', key='stage', mode='w')

In [ ]:
# #Convert .h5 to .csv
# import pandas as pd
# df = pd.read_hdf('goldstein_index_1990_final.h5')
# df.to_csv('goldstein_index_1990_final.csv', index=False)

In [ ]:
#read all csvs mentioned in a list into a single dataframe
for csv in csvs:
    df = pd.read_csv(csv)
    df_final = pd.concat([df_final, df], ignore_index=True)